In [1]:
import os
import torch
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from feature_eng.scalers import ranged_scaler
from datetime import datetime, timedelta
#from mpge.rca import mpge_root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Or if you are using > Python 3.11:
with warnings.catch_warnings(action="ignore"):
    fxn()

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [3]:
from tkgngc.embeddings import PretrainedTKGEmbeddingWithTimestamps
from tkgngc.data_processing import TKGNGCDataProcessor
from tkgngc.model import train_model

In [4]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [5]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [6]:
cats_rows_list = metadata.rows(named=True)
cats_df.head()

timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category,entity_id
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
2023-01-01 00:00:00,0.0,0.571429,0.25,0.5,0.5,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688,0.0,0.0,0
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.40973,0.246024,0.14197,0.112819,0.550194,0.406691,0.0,0.0,1
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698,0.0,0.0,2
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.40671,0.0,0.0,3
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726,0.0,0.0,4


In [7]:
cats_df = cats_df.to_pandas()

In [8]:
potential_causes = metadata['root_cause'].unique().to_list()

In [9]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [10]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [11]:
train_df.head()

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")


In [13]:
device

device(type='cuda')

In [14]:
train_df.columns

Index(['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1'],
      dtype='object')

In [15]:
tkgnc_data = TKGNGCDataProcessor(train_df,device,num_timestamps=20, lags=1)

In [16]:
pretrained_tkg = PretrainedTKGEmbeddingWithTimestamps(
    num_entities=int(tkgnc_data.entity_indices.max().item()+1),
    num_relations=int(tkgnc_data.relation_indices.max().item()+1),
    embedding_dim=17,
    num_timestamps=tkgnc_data.num_timestamps,
).to(device)

In [17]:
quads = (
    tkgnc_data.entity_indices[:-1],  # Head entities
    tkgnc_data.relation_indices,  # Relations
    tkgnc_data.entity_indices[1:],  # Tail entities (shifted example)
    tkgnc_data.timestamp_indices[:-1],  # Timestamps
)


In [18]:
#pretrained_tkg.pretrain(quads, learning_rate=0.01, epochs=1000)
#torch.save(pretrained_tkg.state_dict(), 'pretrained_tkg')
pretrained_tkg.load_state_dict(torch.load('pretrained_tkg'))

<All keys matched successfully>

In [19]:
from CARAT.data_processing import *
from CARAT.model import *
from CARAT.causal_inference import causal_inference

Using device: cuda


C:\Users\jlowh\anaconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [21]:
cols = list(train_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

num_nodes = len(train_df.columns)

new_metadata = []
A0 = get_adjacency(cols,causal_indices,non_causal_indices,num_nodes)    
BATCH_SIZE = 64

model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from utils.utils import set_seed
set_seed()
n_correct = 0
total_checked = 0
for i, row in enumerate(cats_rows_list):
    total_checked +=1 
    print('Model: '+ str(i))
    anomaly = eval(row['affected'])[0]

    anomaly_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    #start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    root_cause = row['root_cause']
    mod_df = test_df[(test_df.index>= anomaly_time) & (test_df.index<= end_time)]
    mod_df = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1']]
    num_nodes = len(mod_df.columns)

    start_len = mod_df.shape[0]
    if start_len >1000:
        start_len = 1000

    start_time = anomaly_time- timedelta(seconds=start_len)

    normal_df = test_df[(test_df.index>= start_time) & (test_df.index< anomaly_time)]

    sample_data = TKGNGCDataProcessor(mod_df,device,num_timestamps=20, lags=1)
    model_data = create_granger_gat_data(pretrained_tkg,sample_data)
    model_data.retrain_tkg()
    features = model_data.time_series_data
    model_name = 'model_category_'+str(row['category'])
    """model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))"""
    model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    data = create_lagged_features(features, 3, pad_value=0)
    ee = create_lagged_features(model_data.entity_emb, 3, pad_value=0)
    tt = create_lagged_features(model_data.timestamp_emb, 3, pad_value=0)
        
    dataset = TimeSeriesDataset(data, ee, tt)
    dataloader = DataLoader(dataset,batch_size = BATCH_SIZE, shuffle=False)
    loss = train_causal_vae(model, optimizer, dataloader, A0, num_epochs=1000)

    model.eval()
    with torch.no_grad():
        _, _, _, learned_adj_matrix = model(data,ee,tt, num_nodes=num_nodes)
        causal_graph = learned_adj_matrix.cpu().numpy()


    adj_data = pd.DataFrame(causal_graph,index=cols,columns=cols)
    candidates = adj_data[anomaly].sort_values(ascending=False)
    candidates = candidates[candidates.index.isin( potential_causes)]
    #A0 = learned_adj_matrix
    try:
        ci = causal_inference(causal_graph, mod_df, normal_df)
        ci.infer_causal_path(anomaly)  

        if len(ci.causal_factors) >= 3:
            potential_cause1 = ci.causal_factors[0][0]
            potential_cause2 = ci.causal_factors[1][0]
            potential_cause3 = ci.causal_factors[2][0]
        elif len(ci.causal_factors) == 2:
            potential_cause1 = ci.causal_factors[0][0]
            potential_cause2 = ci.causal_factors[1][0]
            potential_cause3 = 'NA'
        elif len(ci.causal_factors) == 1:
            potential_cause1 = ci.causal_factors[0][0]
            potential_cause2 = "NA"
            potential_cause3 = 'NA'
        else:
            potential_cause1 = candidates.index[0]
            potential_cause2 = candidates.index[1]
            potential_cause3 = candidates.index[2]
    except:
        potential_cause1 = candidates.index[0]
        potential_cause2 = candidates.index[1]
        potential_cause3 = candidates.index[2]

   
    if root_cause in [potential_cause1,potential_cause2,potential_cause3]:
        n_correct+=1

    if root_cause == potential_cause1:
        row['cause_1'] = 1
    if root_cause == potential_cause2:
        row['cause_2'] = 1
    if root_cause == potential_cause3:
        row['cause_3'] = 1
    new_metadata.append(row)
    if root_cause in [potential_cause1 , potential_cause2 , potential_cause3]:
        print('ROOT CAUSE FOUND!')
    print(root_cause + '-->' + potential_cause1 + ' | ' + potential_cause2 + ' | ' + potential_cause3)

    print("Model Accuracy: " + str(round( (n_correct/total_checked)*100,2)) + '%' )
    
    print('--------------------------------------------------------------------------------------------------------------------------------')



Model: 0
Epoch 0, Loss: 0.0004
Epoch 0: Average Loss = 245.5344, Best Loss = 245.5344
Epoch 100: Average Loss = 42.5929, Best Loss = 42.4835
Epoch 200: Average Loss = 35.7108, Best Loss = 35.7108
Early stopping triggered after 275 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 151/151 [00:01<00:00, 80.50it/s]


ROOT CAUSE FOUND!
bso3-->bfo1 | bso3 | bfo2
Model Accuracy: 100.0%
--------------------------------------------------------------------------------------------------------------------------------
Model: 1
Epoch 0, Loss: 0.0015
Epoch 0: Average Loss = 207.1768, Best Loss = 207.1768
Epoch 100: Average Loss = 33.6059, Best Loss = 33.5947
Early stopping triggered after 149 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 112/112 [00:00<00:00, 729.77it/s]


ROOT CAUSE FOUND!
bso3-->bfo1 | bso1 | bso3
Model Accuracy: 100.0%
--------------------------------------------------------------------------------------------------------------------------------
Model: 2
Epoch 0, Loss: 0.0005
Epoch 0: Average Loss = 309.0756, Best Loss = 309.0756
Epoch 100: Average Loss = 36.0679, Best Loss = 35.8804
Epoch 200: Average Loss = 31.7616, Best Loss = 31.6003
Early stopping triggered after 259 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 114/114 [00:00<00:00, 573.77it/s]


ROOT CAUSE FOUND!
bfo2-->bso3 | bfo1 | bfo2
Model Accuracy: 100.0%
--------------------------------------------------------------------------------------------------------------------------------
Model: 3
Epoch 0, Loss: 0.0003
Epoch 0: Average Loss = 261.7169, Best Loss = 261.7169
Epoch 100: Average Loss = 47.3148, Best Loss = 47.3148
Epoch 200: Average Loss = 31.1169, Best Loss = 31.1169
Epoch 300: Average Loss = 24.9325, Best Loss = 24.9325
Epoch 400: Average Loss = 21.6484, Best Loss = 21.5521
Early stopping triggered after 416 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 51/51 [00:00<00:00, 25495.77it/s]


ROOT CAUSE FOUND!
bed2-->bfo2 | bfo1 | bed2
Model Accuracy: 100.0%
--------------------------------------------------------------------------------------------------------------------------------
Model: 4
Epoch 0, Loss: 0.0001
Epoch 0: Average Loss = 240.3892, Best Loss = 240.3892
Epoch 100: Average Loss = 50.6195, Best Loss = 50.6195


In [23]:
stats = pl.DataFrame(new_metadata)
agg_stats = stats.select(pl.sum("cause_1", "cause_2",'cause_3'))
agg_stats.select(pl.sum_horizontal(pl.all())).item()/stats.shape[0]

0.465

In [23]:
model.train()
prior_adj_matrix = A0.to(device).double()  # Move prior_adj_matrix to device
num_epochs = 50
best_loss = float("inf")  # Initialize best loss as infinity
best_model_state = None  # To store best model parameters
epochs_no_improve = 0  # Counter for early stopping
loss_history = []  # Store epoch losses
batch_loss_history = []  # Store all batch losses

for epoch in range(num_epochs):
    total_loss = 0
    batch_losses = []  # Store losses per batch

    for batch_idx, (x_batch, entity_batch, time_batch) in enumerate(dataloader):
        if x_batch.shape[0]<=x_batch.shape[1]:
            continue

        optimizer.zero_grad()

        # Move data to correct device and convert to float64
        x_batch = x_batch.to(device).double()
        entity_batch = entity_batch.to(device).double()
        time_batch = time_batch.to(device).double()
        break

        # Forward pass
        recon_x, mu, logvar, adj_matrix = model(x_batch, entity_batch, time_batch, num_nodes=x_batch.shape[1])

        # Compute loss
        loss = causal_vae_loss(
            recon_x.double(), x_batch, mu.double(), logvar.double(), adj_matrix.double(), prior_adj_matrix.double()
        )

        loss.backward()
        optimizer.step()

        if torch.isnan(loss).any() or torch.isinf(loss).any():
            print("NaN or Inf detected in loss function!")
            print(f"Recon loss: {recon_loss}, KL loss: {kl_loss}, Sparsity loss: {sparsity_loss}, Acyclicity loss: {acyclicity_loss}, Attention loss: {attention_loss}")
            exit(1)


        total_loss += loss.item()
        batch_losses.append(loss.item())  # Store batch loss

    break

    avg_loss = total_loss / len(dataloader)  # Calculate average loss per epoch
    loss_history.append(avg_loss)  # Store epoch loss
    batch_loss_history.append(batch_losses)  # Store batch losses

    # Check if loss improved
    if avg_loss < best_loss:
        best_loss = avg_loss
        best_model_state = copy.deepcopy(model.state_dict())  # Save best model
        epochs_no_improve = 0  # Reset counter
    else:
        epochs_no_improve += 1  # Increment counter if no improvement
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Average Loss = {avg_loss:.4f}, Best Loss = {best_loss:.4f}")

    # Early stopping condition
    if epochs_no_improve >= patience:
        print(f"Early stopping triggered after {epoch} epochs. Restoring best model.")
        model.load_state_dict(best_model_state)  # Restore best model state
        break

In [23]:

import torch
from torch_geometric.utils import dense_to_sparse
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GATv2Conv, GATConv
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch.distributions.normal import Normal
from CARAT.utils import TGCN, A3TGCN
import copy
from utils.utils import set_seed
set_seed()

# Automatically select GPU if available, otherwise use CPU
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device( "cpu")
print(f"Using device: {device}")

class GraphLearningModule(nn.Module):
    """
    Learns an adjacency matrix with Graph Attention.
    """
    def __init__(self, num_nodes, hidden_dim, prior_adj_matrix=None, attention_heads=4):
        super(GraphLearningModule, self).__init__()
        self.num_nodes = num_nodes
        self.attention_heads = attention_heads

        # Learnable adjacency matrix (Move to device)
        self.edge_score = nn.Parameter(torch.randn(num_nodes, num_nodes, dtype=torch.float64, device=device))

        # Prior adjacency matrix (Move to device)
        if prior_adj_matrix is not None:
            self.prior_adj = torch.tensor(prior_adj_matrix, dtype=torch.float64, device=device)
        else:
            self.prior_adj = torch.zeros(num_nodes, num_nodes, dtype=torch.float64, device=device)

    def forward(self):
        adj_matrix = torch.sigmoid(self.edge_score) + self.prior_adj
        adj_matrix = torch.clamp(adj_matrix, 0, 1)
    
        # Convert adjacency matrix to sparse format
        edge_index, edge_weights = dense_to_sparse(adj_matrix)
    
        # Get the true number of nodes (max node index + 1)
        actual_num_nodes = edge_index.max().item() + 1
    
        # Ensure all indices are within bounds
        valid_mask = (edge_index[0] < actual_num_nodes) & (edge_index[1] < actual_num_nodes)
        edge_index = edge_index[:, valid_mask]
        edge_weights = edge_weights[valid_mask]
    
        #print(f"GraphLearningModule Output: edge_index max {edge_index.max()}, expected num_nodes {actual_num_nodes}")
    
        return edge_index.to(device).long(), edge_weights.to(device).double()
    
            

class CausalGraphVAE(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim, latent_dim, num_nodes, prior_adj_matrix=None, attention_heads=4):
        super(CausalGraphVAE, self).__init__()

        # Graph Learning with Attention (Move to device)
        self.graph_learner = GraphLearningModule(num_nodes, hidden_dim, prior_adj_matrix, attention_heads).to(device)

        # Embedding layers for additional inputs
        self.entity_embed_layer = nn.Linear(embed_dim, hidden_dim, dtype=torch.float64).to(device)
        self.timestamp_embed_layer = nn.Linear(embed_dim, hidden_dim, dtype=torch.float64).to(device)

        # Temporal Graph ConvolutionalNetwork
        self.tgcn1 = A3TGCN(input_dim + 2 * hidden_dim, hidden_dim,periods=3).to(device).double()
        self.tgcn2 = A3TGCN(hidden_dim, hidden_dim,periods=3).to(device).double()

        # Latent Space
        self.mu_layer = nn.Linear(hidden_dim, latent_dim, dtype=torch.float64).to(device)
        self.logvar_layer = nn.Linear(hidden_dim, latent_dim, dtype=torch.float64).to(device)

        # Decoder with Temporal Graph ConvolutionalNetwork
        self.decoder_fc = nn.Linear(latent_dim, hidden_dim, dtype=torch.float64).to(device)
        self.tgcn_decoder = TGCN(hidden_dim, input_dim).to(device).double()

    def encode(self, x, entity_emb, time_emb, edge_index, edge_weights):
        """
        Encoding with concatenation of entity and timestamp embeddings.
        """
        # Move inputs to correct device and convert to float64
        x = x.to(device).double()
        entity_emb = entity_emb.to(device).double()
        time_emb = time_emb.to(device).double()
        edge_index = edge_index.to(device).long()
        edge_weights = edge_weights.to(device).double()

        # Transform entity & timestamp embeddings
        entity_emb = F.relu(self.entity_embed_layer(entity_emb))
        time_emb = F.relu(self.timestamp_embed_layer(time_emb))

        # Concatenate embeddings with raw features
        x = torch.cat([x, entity_emb, time_emb], dim=-1)

        # Temporal Graph Convolutional Network Encoding (Pass edge_index dynamically)
        x = F.relu(self.tgcn1(x, edge_index, edge_weights))
       # x = F.relu(self.tgcn2(x, edge_index, edge_weights))

        mu = self.mu_layer(x)
        logvar = self.logvar_layer(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std, device=device, dtype=torch.float64)
        return mu + eps * std

    def decode(self, z, edge_index, edge_weights, num_nodes):
        """
        Decodes the learned representation back to the original space.
        """
        x = self.decoder_fc(z)
        x = F.relu(self.tgcn_decoder(x, edge_index, edge_weights))
        return x

    def forward(self, x, entity_emb, time_emb, num_nodes):
        edge_index, edge_weights = self.graph_learner()
        
        actual_num_nodes = x.shape[2]  # Ensure alignment
        if edge_index.max() >= actual_num_nodes:
            raise ValueError(f"Invalid edge_index detected in CausalGraphVAE! Max index: {edge_index.max()}, Expected < {actual_num_nodes}")
    
        mu, logvar = self.encode(x, entity_emb, time_emb, edge_index, edge_weights)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, edge_index, edge_weights, actual_num_nodes)  # Pass correct num_nodes
    
        return recon_x, mu, logvar, edge_weights.view(actual_num_nodes, actual_num_nodes)


def causal_vae_loss(recon_x, x, mu, logvar, adj_matrix, prior_adj_matrix, 
                    lambda_sparsity=1e-3, lambda_acyclic=1e-2, lambda_attention=1e-2):
    """
    Computes the loss function for the Causal Graph VAE with attention.
    """
    # Move all tensors to the correct device and convert to float64
    recon_x = recon_x.to(device).double()
    x = x.to(device).double()
    mu = mu.to(device).double()
    logvar = logvar.to(device).double()
    adj_matrix = adj_matrix.to(device).double()
    prior_adj_matrix = prior_adj_matrix.to(device).double()

    # Reconstruction loss
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')

    # KL Divergence loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Graph Sparsity loss (L1 regularization)
    sparsity_loss = lambda_sparsity * torch.norm(adj_matrix, p=1)

    # Acyclicity Constraint (Ensures DAG structure)
    eye_matrix = torch.eye(adj_matrix.shape[0], dtype=torch.float64, device=device)  # Fix: Ensure eye_matrix is float64
    H = torch.matrix_exp(adj_matrix * adj_matrix)  # Exponentiate the adjacency matrix
    acyclicity_loss = lambda_acyclic * torch.trace(H - eye_matrix)

    # Attention-based Causal Alignment Loss
    attention_loss = lambda_attention * F.mse_loss(adj_matrix, prior_adj_matrix)

    # Compute total loss
    total_loss = recon_loss + kl_loss + sparsity_loss + acyclicity_loss + attention_loss

    return total_loss


def train_causal_vae(model, optimizer, dataloader, prior_adj_matrix, num_epochs=100, patience=10):
    model.train()
    prior_adj_matrix = prior_adj_matrix.to(device).double()  # Move prior_adj_matrix to device

    best_loss = float("inf")  # Initialize best loss as infinity
    best_model_state = None  # To store best model parameters
    epochs_no_improve = 0  # Counter for early stopping
    loss_history = []  # Store epoch losses
    batch_loss_history = []  # Store all batch losses

    for epoch in range(num_epochs):
        total_loss = 0
        batch_losses = []  # Store losses per batch

        for batch_idx, (x_batch, entity_batch, time_batch) in enumerate(dataloader):
            if x_batch.shape[0]<=x_batch.shape[1]:
                continue

            optimizer.zero_grad()

            # Move data to correct device and convert to float64
            x_batch = x_batch.to(device).double()
            entity_batch = entity_batch.to(device).double()
            time_batch = time_batch.to(device).double()

            # Forward pass
            recon_x, mu, logvar, adj_matrix = model(x_batch, entity_batch, time_batch, num_nodes=x_batch.shape[1])

            # Compute loss
            loss = causal_vae_loss(
                recon_x.double(), x_batch, mu.double(), logvar.double(), adj_matrix.double(), prior_adj_matrix.double()
            )

            loss.backward()
            optimizer.step()

            if torch.isnan(loss).any() or torch.isinf(loss).any():
                print("NaN or Inf detected in loss function!")
                print(f"Recon loss: {recon_loss}, KL loss: {kl_loss}, Sparsity loss: {sparsity_loss}, Acyclicity loss: {acyclicity_loss}, Attention loss: {attention_loss}")
                exit(1)


            total_loss += loss.item()
            batch_losses.append(loss.item())  # Store batch loss

        avg_loss = total_loss / len(dataloader)  # Calculate average loss per epoch
        loss_history.append(avg_loss)  # Store epoch loss
        batch_loss_history.append(batch_losses)  # Store batch losses

        # Check if loss improved
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_model_state = copy.deepcopy(model.state_dict())  # Save best model
            epochs_no_improve = 0  # Reset counter
        else:
            epochs_no_improve += 1  # Increment counter if no improvement
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Average Loss = {avg_loss:.4f}, Best Loss = {best_loss:.4f}")

        # Early stopping condition
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch} epochs. Restoring best model.")
            model.load_state_dict(best_model_state)  # Restore best model state
            break

    return loss_history  # Return trained model, epoch losses, and batch losses

Using device: cpu


In [25]:
prior_adj_matrix = A0
num_epochs=100
patience=10


model.train()
prior_adj_matrix = prior_adj_matrix.to(device).double()  # Move prior_adj_matrix to device

best_loss = float("inf")  # Initialize best loss as infinity
best_model_state = None  # To store best model parameters
epochs_no_improve = 0  # Counter for early stopping
loss_history = []  # Store epoch losses
batch_loss_history = []  # Store all batch losses

for epoch in range(num_epochs):
    total_loss = 0
    batch_losses = []  # Store losses per batch

    for batch_idx, (x_batch, entity_batch, time_batch) in enumerate(dataloader):
        if x_batch.shape[0]<=x_batch.shape[1]:
            continue

        optimizer.zero_grad()

        # Move data to correct device and convert to float64
        x_batch = x_batch.to(device).double()
        entity_batch = entity_batch.to(device).double()
        time_batch = time_batch.to(device).double()

        # Forward pass
        recon_x, mu, logvar, adj_matrix = model(x_batch, entity_batch, time_batch, num_nodes=x_batch.shape[1])

        # Compute loss
        loss = causal_vae_loss(
            recon_x.double(), x_batch, mu.double(), logvar.double(), adj_matrix.double(), prior_adj_matrix.double()
        )


RuntimeError: The size of tensor a (64) must match the size of tensor b (3) at non-singleton dimension 1

(17, 17)

In [ ]:
recon_x = recon_x.to(device).double()
x = x.to(device).double()
mu = mu.to(device).double()
logvar = logvar.to(device).double()
adj_matrix = adj_matrix.to(device).double()
prior_adj_matrix = prior_adj_matrix.to(device).double()

# Reconstruction loss
recon_loss = F.mse_loss(recon_x, x, reduction='sum')

# KL Divergence loss
kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# Graph Sparsity loss (L1 regularization)
sparsity_loss = lambda_sparsity * torch.norm(adj_matrix, p=1)

# Acyclicity Constraint (Ensures DAG structure)
eye_matrix = torch.eye(adj_matrix.shape[0], dtype=torch.float64, device=device)  # Fix: Ensure eye_matrix is float64
H = torch.matrix_exp(adj_matrix * adj_matrix)  # Exponentiate the adjacency matrix
acyclicity_loss = lambda_acyclic * torch.trace(H - eye_matrix)

# Attention-based Causal Alignment Loss
attention_loss = lambda_attention * F.mse_loss(adj_matrix, prior_adj_matrix)

# Compute total loss
total_loss = recon_loss + kl_loss + sparsity_loss + acyclicity_loss + attention_loss

In [27]:
recon_x.shape

torch.Size([64, 17])

In [28]:
x_batch[:,0,:]

torch.Size([64, 17])

In [37]:
input_dim=num_nodes
hidden_dim=128
latent_dim=16
embed_dim=17
prior_adj_matrix=A0.to(torch.float)
attention_heads=4
ema_decay=0.9


graph_learner = GraphLearningModule(num_nodes, hidden_dim, prior_adj_matrix, attention_heads).to(device)

# Embedding layers
entity_embed_layer = nn.Linear(embed_dim, hidden_dim, dtype=torch.float64).to(device)
timestamp_embed_layer = nn.Linear(embed_dim, hidden_dim, dtype=torch.float64).to(device)

# Temporal Graph Convolutional Network
tgcn1 = A3TGCN(input_dim + 2 * hidden_dim, hidden_dim,periods=3).to(device).double()
tgcn2 = A3TGCN(hidden_dim, hidden_dim,periods=3).to(device).double()

# Graph-GRU for Temporal State Retention
graph_memory = nn.GRU(hidden_dim, hidden_dim, batch_first=False).to(device).double()

# Latent Space
mu_layer = nn.Linear(hidden_dim, latent_dim, dtype=torch.float64).to(device)
logvar_layer = nn.Linear(hidden_dim, latent_dim, dtype=torch.float64).to(device)

# Decoder
decoder_fc = nn.Linear(latent_dim, hidden_dim,dtype=torch.float64).to(device)
tgcn_decoder = TGCN(hidden_dim, input_dim).to(device).double()

In [38]:
edge_index, edge_weights = graph_learner()
x = x_batch.to(device).double()
entity_emb = entity_batch.to(device).double()
time_emb = time_batch.to(device).double()
edge_index = edge_index.to(device).long()
edge_weights = edge_weights.to(device).double()

# Transform entity & timestamp embeddings
entity_emb = F.relu(entity_embed_layer(entity_emb))
time_emb = F.relu(timestamp_embed_layer(time_emb))

# Concatenate embeddings with raw features
x = torch.cat([x, entity_emb, time_emb], dim=-1)

# Temporal Graph Convolutional Network Encoding (Pass edge_index dynamically)
x = F.relu(tgcn1(x, edge_index, edge_weights))
#x = F.relu(tgcn2(x, edge_index, edge_weights))

mu = mu_layer(x)
logvar = logvar_layer(x)

In [39]:
def reparameterize( mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std, device=device, dtype=torch.float64)
    return mu + eps * std

In [40]:
z = reparameterize(mu, logvar)

In [41]:
x = decoder_fc(z)


In [42]:
x.shape

torch.Size([64, 128])

In [43]:
x = F.relu(tgcn_decoder(x, edge_index, edge_weights))

In [27]:

in_channels = input_dim
out_channels = 2 * hidden_dim
periods = 3
improved= False
cached = False
add_self_loops = True


_base_tgcn = TGCN(
    in_channels=in_channels,
    out_channels=out_channels,
    improved=improved,
    cached=cached,
    add_self_loops=add_self_loops,

).to(device)
device = torch.device('cpu')
_attention = torch.empty(periods, device=device)
torch.nn.init.uniform_(_attention)

tensor([0.7049, 0.5933, 0.3940])

In [30]:
X = x.to(device)
H = hidden_state.to(device)
edge_weight=edge_weights.to(device)
edge_index = edge_index.to(device)

H_accum = 0
probs = torch.nn.functional.softmax(_attention, dim=0).to(device)
for period in range(3):
    H_accum = H_accum + probs[period] * _base_tgcn(
        X[:, period, :], edge_index, edge_weight, H
        #X[:, period, :], edge_index, edge_weight, H
    )


RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [31]:
H = _base_tgcn._set_hidden_state(X[:, period,:], H)

In [33]:
H.shape

torch.Size([1, 64, 128])

In [32]:
Z = _base_tgcn._calculate_update_gate(X[:, period,:], edge_index, edge_weight, H)

RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [34]:
from torch_geometric.nn import GCNConv

conv_z = GCNConv(
    in_channels=in_channels,
    out_channels=out_channels,
    improved=improved,
    cached=cached,
    add_self_loops=add_self_loops,
)

linear_z = torch.nn.Linear(2 * out_channels, out_channels)

In [41]:
torch.unsqueeze(X[:,period,:],0)

torch.Size([1, 64, 17])

In [39]:
H.shape

torch.Size([1, 64, 128])

In [45]:
Z = torch.cat([conv_z(torch.unsqueeze(X[:,period,:],0), edge_index, edge_weight), H], axis=2)

In [46]:
Z.shape

torch.Size([1, 64, 384])

In [48]:

Z = linear_z(Z)
Z = torch.sigmoid(Z)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x384 and 512x256)

In [ ]:
H = self._set_hidden_state(X, H)
Z = self._calculate_update_gate(X, edge_index, edge_weight, H)
R = self._calculate_reset_gate(X, edge_index, edge_weight, H)
H_tilde = self._calculate_candidate_state(X, edge_index, edge_weight, H, R)
H = self._calculate_hidden_state(Z, H, H_tilde)

In [ ]:
improved = False
cached = False
add_self_loops= True


 def _create_update_gate_parameters_and_layers():

        conv_z = GCNConv(
            in_channels=in_channels,
            out_channels=out_channels,
            improved=improved,
            cached=cached,
            add_self_loops=add_self_loops,
        )

        linear_z = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_reset_gate_parameters_and_layers(self):

        self.conv_r = GCNConv(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            improved=self.improved,
            cached=self.cached,
            add_self_loops=self.add_self_loops,
        )

        self.linear_r = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_candidate_state_parameters_and_layers(self):

        self.conv_h = GCNConv(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            improved=self.improved,
            cached=self.cached,
            add_self_loops=self.add_self_loops,
        )

        self.linear_h = torch.nn.Linear(2 * self.out_channels, self.out_channels)

    def _create_parameters_and_layers(self):
        self._create_update_gate_parameters_and_layers()
        self._create_reset_gate_parameters_and_layers()
        self._create_candidate_state_parameters_and_layers()

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _calculate_update_gate(self, X, edge_index, edge_weight, H):
        Z = torch.cat([self.conv_z(X, edge_index, edge_weight), H], axis=1)
        Z = self.linear_z(Z)
        Z = torch.sigmoid(Z)
        return Z

    def _calculate_reset_gate(self, X, edge_index, edge_weight, H):
        R = torch.cat([self.conv_r(X, edge_index, edge_weight), H], axis=1)
        R = self.linear_r(R)
        R = torch.sigmoid(R)
        return R

    def _calculate_candidate_state(self, X, edge_index, edge_weight, H, R):
        H_tilde = torch.cat([self.conv_h(X, edge_index, edge_weight), H * R], axis=1)
        H_tilde = self.linear_h(H_tilde)
        H_tilde = torch.tanh(H_tilde)
        return H_tilde

    def _calculate_hidden_state(self, Z, H, H_tilde):
        H = Z * H + (1 - Z) * H_tilde
        return H



In [ ]:
    def forward(
        self,
        X: torch.FloatTensor,
        edge_index: torch.LongTensor,
        edge_weight: torch.FloatTensor = None,
        H: torch.FloatTensor = None,
    ) -> torch.FloatTensor:
        """
        Making a forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph. If the hidden state matrix is not present
        when the forward pass is called it is initialized with zeros.

        Arg types:
            * **X** *(PyTorch Float Tensor)* - Node features.
            * **edge_index** *(PyTorch Long Tensor)* - Graph edge indices.
            * **edge_weight** *(PyTorch Long Tensor, optional)* - Edge weight vector.
            * **H** *(PyTorch Float Tensor, optional)* - Hidden state matrix for all nodes.

        Return types:
            * **H** *(PyTorch Float Tensor)* - Hidden state matrix for all nodes.
        """
        H = self._set_hidden_state(X, H)
        Z = self._calculate_update_gate(X, edge_index, edge_weight, H)
        R = self._calculate_reset_gate(X, edge_index, edge_weight, H)
        H_tilde = self._calculate_candidate_state(X, edge_index, edge_weight, H, R)
        H = self._calculate_hidden_state(Z, H, H_tilde)
        return H